# Türk Hukuk RAG Chatbot

Bu notebook, `Renicames/turkish-law-chatbot` veri seti ile RAG boru hattının uçtan uca nasıl kurulduğunu örnekler.


In [ ]:
!pip -q install -r ../requirements.txt


### Alternatif: BuyukSinema'yı doğrudan yükle ve pandas'a çevir


In [ ]:
from datasets import load_dataset
import pandas as pd
import re

# BuyukSinema datasetini HF Datasets ile yükle
# Not: Bu veri setinde text ve label kolonları bulunur
buyuk = load_dataset("turkish-nlp-suite/BuyukSinema", split="train")
print(buyuk)
print(buyuk.column_names)
print(buyuk[0])

# Pandas DataFrame'e çevir
df = buyuk.to_pandas()
print(df.head())

# Basit bir temizlik örneği (isteğe bağlı)
def basic_clean(t: str) -> str:
    if not isinstance(t, str):
        return ""
    t = re.sub(r"\s+", " ", t).strip()
    return t

df["text_clean"] = df["text"].map(basic_clean)
df.head()


### (Opsiyonel) Haystack Document formatına dönüştürme


In [ ]:
# Eğer Haystack kullanacaksanız şu örnek işinize yarar
# pip install farm-haystack
try:
    from haystack.dataclasses import Document
    docs = [
        Document(content=row["text_clean"], meta={"label": int(row["label"])})
        for _, row in df.iterrows()
        if isinstance(row["text_clean"], str) and len(row["text_clean"]) > 0
    ]
    print(docs[0] if docs else "no docs")
except Exception as e:
    print("Haystack kurulu değil veya Document importu başarısız:", e)


## 1) Veri Setini Yükle


In [ ]:
from datasets import load_dataset

ds = load_dataset('turkish-nlp-suite/BuyukSinema')
ds


## 2) FAISS İndeksi Oluştur (script)


In [ ]:
%run ../src/build_index.py


## 3) Soru Sor ve Yanıt Al


In [ ]:
from src.rag_pipeline import answer_question

resp = answer_question('Duygu olarak en olumlu yorumlar nelerden bahsediyor?', top_k=5)
resp['answer']
